# Real-time Video

In [ ]:
import cv2

inputVideo = cv2.VideoCapture(0)
#print(type(inputVideo))
#check if connection with camera is successfully
if inputVideo.isOpened():
    ret, frame = inputVideo.read()  #capture a frame from live video

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

# You can read camera parameters from tutorial_camera_params.yml
fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

# Set coordinate system
objPoints = np.array([[-markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, -markerLength/2.0, 0], [-markerLength/2.0, -markerLength/2.0, 0]], dtype=np.float32)

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.drawFrameAxes(frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)
    cv2.imshow("out", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

inputVideo.release()
cv2.destroyAllWindows()

## Video Input

### Intersection mean

In [1]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders_Trim.mp4")

if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.5
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        for id, rvec, tvec in zip(ids, rvecs, tvecs):
            cv2.drawFrameAxes(scaled_frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)

            if len(tvecs) == 3:
                if tuple(id) in prev_positions:
                    prev_t, prev_p, prev_o = prev_positions[tuple(id)]
                    dt = pd.Timestamp.now() - prev_t
                    v = (tvec - prev_p) / dt.total_seconds()
                    p = tvec + v*dt.total_seconds()
                    o = rvec + (rvec - prev_o) / dt.total_seconds()
                else:
                    p, o = tvec, rvec

                intersection = np.mean(tvecs, axis=0)
                print(ids, intersection)
                rvec = np.zeros((3,1))
                tvec = np.zeros((3,1))
                imgpts, _ = cv2.projectPoints(intersection, rvec, tvec, cameraMatrix, distCoeffs)

                # draw circle at intersection point
                center = tuple(map(int, imgpts[0].ravel()))
                cv2.circle(scaled_frame, center, 5, (0, 255, 0), -1)
                #print('p:', p, 'o:', o)
                data = {'Time': pd.Timestamp.now(), 'Marker ID': int(id), 'tx': p[0][0], 'ty': p[0][1], 'tz': p[0][2], 'rx': o[0][0], 'ry': o[0][1], 'rz': o[0][2]}
                pose_data = pose_data.append(data, ignore_index=True)
                prev_positions[tuple(id)] = (pd.Timestamp.now(), p, o)

    cv2.imshow("out", scaled_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

pose_data.to_excel("pose_data.xlsx", index=False)
inputVideo.release()
cv2.destroyAllWindows()

True
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47404637 -0.0295659   1.42979853]]
[[ 3]
 [13]
 [11]] [[-0.47757406 -0.02994701  1.43863444]]
[[ 3]
 [13]
 [11]] [[-0.47757406 -0.02994701  1.43863444]]
[[ 3]
 [13]
 [11]] [[-0.47757406 -0.02994701  1.43863444]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.43175388]]
[[ 3]
 [13]
 [11]] [[-0.47461888 -0.02951183  1.431

### ids

### x-axis

Notes:
`drawDetectedMarkers`
This function draws the axes of the world/object coordinate system w.r.t. to the camera frame. OX is drawn in red, OY in green and OZ in blue.

In [9]:
def generate_second_point(rvec, tvec, axis_distance=1):
    # Convert rvec to rotation matrix
    rot_mat, _ = cv2.Rodrigues(rvec)
    
    # Extract the x-axis from the rotation matrix
    x_axis = rot_mat[:, 0]
    
    # Scale the x-axis by the desired distance and add it to the origin
    second_point = tvec + x_axis * axis_distance
    
    return second_point

In [30]:
def intersect_pnt(P11,P12,P21,P22):
  # Input values
  x1 = P11[0]
  x2 = P21[0]
  y1 = P11[1]
  y2 = P21[1]
  z1 = P11[2]
  z2 = P21[2]
  # direction vectors
  l1 = P12[0]-P11[0]
  m1 = P12[1]-P11[1]
  n1 = P12[2]-P11[2]

  l2 = P22[0]-P21[0]
  m2 = P22[1]-P21[1]
  n2 = P22[2]-P21[2]

  # Define the coefficient matrix and the constant vector
  A = np.array([[l1, -l2],
                [m1, -m2],
                [n1, -n2]])
  B = np.array([x2- x1, y2- y1, z2 - z1]).reshape(3,1)
  # Solve for t and s
  try:
    solution = np.linalg.pinv(A) @ B
    #print(solution)
    # Extract the values of t and s
    t = solution[0]
    s = solution[1]

    # Print the solution
    print("The solution for t is: {}".format(t))
    print("The solution for s is: {}".format(s))

    L1 = np.array([x1+l1*t, y1+m1*t, z1+n1*t])
    L2 = np.array([x2+l2*s, y2+m2*s, z2+n2*s])
    #print("intersection:", L1)

    #if L2 == L1:
    #  print("Lines intersect")
      
  except :
    print("error")

  return L1


In [31]:

P11 = [1,1,4]
P12 = [0,0,0]

P21 = [0.5,0.5,1]
P22 = [0.5,0.5,3]

intersect_pnt(P11,P12,P21,P22)  

The solution for t is: [0.5]
The solution for s is: [0.5]


array([[0.5],
       [0.5],
       [2. ]])

In [35]:
def find_line_equation(point1, point2):
    # Extract x and y coordinates from the numpy arrays
    x1, y1, z1 = point1[:2]
    x2, y2, z2 = point2[:2]
    l = (x2 - x1)
    m = (y2 - y1)
    n = (z2 - z1)
    
    # Calculate the slope of the line
    if x2 - x1 == 0:
        slope = float('inf')
    else:
       
        slope = (y2 - y1) / (x2 - x1) 

    # Calculate the y-intercept of the line
    y_intercept = y1 - slope * x1

    return slope, y_intercept

In [3]:
def find_intersection(slope1, y_intercept1, slope2, y_intercept2):
    if slope1 == slope2:
        return None
    else:
        x = (y_intercept2 - y_intercept1) / (slope1 - slope2)
        y = slope1 * x + y_intercept1
        intersection = np.array([x, y])
        if np.isnan(intersection).any() or np.isinf(intersection).any():
            print("Intersection point is NaN or infinity, skipping...")
            return None
        else:
            return intersection

In [23]:
import numpy as np
import cv2
dist = 0.5
origin = np.array([0, 0, 0])
point0 = origin.flatten()

#first line
rvec1 = np.array([-0.32230617, -3.14875594, -0.74449442])
tvec1 = np.array([-0.37034921,  0.08955338,  3.79995547])
point1 = generate_second_point(rvec1, tvec1, dist).flatten()

#second line
rvec2 = np.array([-1.0717847 ,  2.47989969,  0.43640154])
tvec2 = np.array([-0.98645196,  0.70029736,  4.19780266])
point2 = tvec2.flatten()

point3 = generate_second_point(rvec2, tvec2, dist).flatten()

slope1, y_intercept1 = find_line_equation(point0, point1)
slope2, y_intercept2 = find_line_equation(point2, point3)

print(f"Line 1 equation: y = {slope1}x + {y_intercept1}")
print(f"Line 2 equation: y = {slope2}x + {y_intercept2}")

[-0.85753195  0.19781828  3.76943186]
Line 1 equation: y = -0.23068327484633158x + 0.0
Line 2 equation: y = 0.9896425067436612x + 1.6765321504765978


In [6]:
# Find intersection of two lines
intersection = find_intersection(slope1, y_intercept1, slope2, y_intercept2)
print(intersection) 

[-1.37383982  0.31692187]


In [7]:
def find_intersection_3d(slope1, y_intercept1, slope2, y_intercept2):
    # Calculate the direction vectors of the lines
    v1 = np.array([1, slope1, slope1*y_intercept1])
    v2 = np.array([1, slope2, slope2*y_intercept2])
    
    # Calculate the cross product of the direction vectors
    cross_product = np.cross(v1, v2)
    
    if np.allclose(cross_product, 0):
        # Lines are parallel, so no intersection
        return None
    
    # Calculate the point on each line
    point1 = np.array([0, y_intercept1, 0])
    point2 = np.array([0, y_intercept2, 0])
    
    # Calculate the dot products
    dot_product1 = np.dot(cross_product, point1)
    dot_product2 = np.dot(cross_product, point2)
    
    # Calculate the intersection point
    t1 = dot_product2 / np.dot(cross_product, v1)
    intersection_point = point1 + t1 * v1
    
    if np.any(np.isnan(intersection_point)) or np.any(np.isinf(intersection_point)):
        print("Intersection point is NaN or infinity")
        return None
    
    return intersection_point

In [27]:
dist = 1

#first line
rvec1 = np.array([2.28203153, -1.0458793, -1.47315036])
tvec1 = np.array([0.68467662, 0.03170761, 3.6583422]) #0.68467662 0.03170761 3.6583422

point0 = tvec1
point1 = generate_second_point(rvec1, tvec1, dist).squeeze()
print(point0)
print(point1)
#second line
rvec2 = np.array([3.18584118, -0.69075018, -0.21338737])
tvec2 = np.array([0.09015421, 0.61260409, 3.88713185])

point2 = tvec2
point3 = generate_second_point(rvec2, tvec2, dist).squeeze()

slope1, y_intercept1 = find_line_equation(point0, point1)
slope2, y_intercept2 = find_line_equation(point2, point3)

print(f"Line equation: y = {slope1}x + {y_intercept1}")
print(f"Line equation: y = {slope2}x + {y_intercept2}")


[0.68467662 0.03170761 3.6583422 ]
[ 0.9243347  -0.64013585  2.95749245]
Line equation: y = -2.8033415192161026x + 1.951090006082546
Line equation: y = -0.4461546468240358x + 0.65282680972225


In [21]:
intersection3d = find_intersection_3d(slope1, y_intercept1, slope2, y_intercept2)
print(intersection3d)

Intersection point is NaN or infinity
None


<ipython-input-7-54ea53270e44>:22: RuntimeWarning: divide by zero encountered in double_scalars
  t1 = dot_product2 / np.dot(cross_product, v1)


In [36]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders_Trim_Trim.mp4")
if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.2

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}
while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.7
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        
        detected_ids = set(ids.flatten())
        if 12 in detected_ids and 5 in detected_ids:  # check for required ids
            marker_1_index = np.where(ids == 12)[0][0]
            marker_2_index = np.where(ids == 5)[0][0]
            marker_1 = tvecs[marker_1_index]
            marker_2 = tvecs[marker_2_index]
            
            print("marker1", marker_1,"marker2", marker_2)
            #print('rvec marker1', rvecs[marker_1_index])
            #print('rvec marker2', rvecs[marker_2_index])

            # find the intersection point
            marker_1_x = -marker_1[0][0]
            marker_2_x = marker_2[0][0]
            
            point0 = marker_1.flatten()
            point1 = generate_second_point(rvecs[marker_1_index], tvecs[marker_1_index]).flatten()
            point2 = marker_2.flatten()
            point3 = generate_second_point(rvecs[marker_2_index], tvecs[marker_2_index]).flatten()
            '''
            print('point0: ',point0)
            print('point1: ',point1)
            print('point2: ',point2)
            print('point3: ',point3)
            
            slope1, y_intercept1 = find_line_equation(point0, point1)
            slope2, y_intercept2 = find_line_equation(point2, point3)
            '''
            intersection = intersect_pnt(point0, point1, point2, point3)
            print("intersection: ", intersection)

            # Draw a circle at the second point (assuming we have an image called 'scaled_frame')
            '''
            second_point_image = cv2.projectPoints(point1, rvecs[marker_2_index], tvecs[marker_2_index], cameraMatrix, distCoeffs)[0][0]

            print("ids: ",ids, end=' ')
            print("intersection shape: ", intersection.shape)
            print("intersection type: ", intersection.dtype)
            '''
            if intersection is not None:
                image_points = cv2.projectPoints(intersection, rvecs[marker_1_index], tvecs[marker_1_index], cameraMatrix, distCoeffs)[0][0]
                #print("image_points",image_points)
                

                # Draw a circle at the intersection point (assuming we have an image called 'frame')
                cv2.circle(scaled_frame, (int(image_points[0][0]), int(image_points[0][1])) , 5, (255, 0, 255), -1)
            
        # Draw the detected aruco markers and axes
        ee = cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        for i in range(len(ids)):
            ee = cv2.drawFrameAxes(ee, cameraMatrix, distCoeffs, rvecs[i], tvecs[i], 0.5, thickness=2)
            

    # Display the resulting frame
    cv2.imshow('frame', ee)


    # Exit loop if 'esc' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break
# Release the video capture object and close all windows
inputVideo.release()
cv2.destroyAllWindows()

True
marker1 [[0.68467662 0.03170761 3.6583422 ]] marker2 [[0.09015421 0.61260409 3.88713185]]
The solution for t is: [-0.331529]
The solution for s is: [0.60847423]
intersection:  [[0.60522302]
 [0.25444321]
 [3.89069422]]
marker1 [[0.70624544 0.02978499 3.66179038]] marker2 [[0.1110463  0.60086494 3.82932258]]
The solution for t is: [-0.45138905]
The solution for s is: [0.56999875]
intersection:  [[0.59985551]
 [0.33339025]
 [3.97842441]]
marker1 [[0.70624544 0.02978499 3.66179038]] marker2 [[0.1110463  0.60086494 3.82932258]]
The solution for t is: [-0.45138905]
The solution for s is: [0.56999875]
intersection:  [[0.59985551]
 [0.33339025]
 [3.97842441]]
marker1 [[0.70624544 0.02978499 3.66179038]] marker2 [[0.1110463  0.60086494 3.82932258]]
The solution for t is: [-0.45138905]
The solution for s is: [0.56999875]
intersection:  [[0.59985551]
 [0.33339025]
 [3.97842441]]
marker1 [[0.7212786  0.02942668 3.6228923 ]] marker2 [[0.13112144 0.60102473 3.83054903]]
The solution for t is: 

In [ ]:
rvecs

array([[[-0.32230617, -3.14875594, -0.74449442]],

       [[-1.0717847 ,  2.47989969,  0.43640154]],

       [[-2.20132837,  0.70330097, -0.6320501 ]],

       [[-1.6409619 , -2.28496544, -0.73582845]],

       [[ 2.03470912,  0.0729016 ,  0.51532167]]])

### y-axis

In [ ]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders.mp4")

if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.2

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}
while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.5
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        detected_ids = set(ids.flatten())
        '''if (12 in detected_ids and 5 in detected_ids) or (13 in detected_ids and 1 in detected_ids):  # check for required ids
            if 12 in detected_ids and 5 in detected_ids:
                marker_1_index = np.where(ids == 12)[0][0]
                marker_2_index = np.where(ids == 5)[0][0]
            else:
                marker_1_index = np.where(ids == 13)[0][0]
                marker_2_index = np.where(ids == 1)[0][0]
            marker_1 = tvecs[marker_1_index]
            marker_2 = tvecs[marker_2_index]
            
            print("marker1", marker_1,"marker2", marker_2)
            # find the intersection point
            marker_1_x = -marker_1[0][0]
            marker_2_x = marker_2[0][0]
            intersection_y = ((marker_2_x - marker_1_x) * marker_1[0][1] + (marker_1_x * marker_2[0][1] - marker_2_x * marker_1[0][1])) / (marker_2_x - marker_1_x)
            intersection_z = ((marker_2_x - marker_1_x) * marker_1[0][2] + (marker_1_x * marker_2[0][2] - marker_2_x * marker_1[0][2])) / (marker_2_x - marker_1_x)
            intersection = np.array([(marker_1_x + marker_2_x) / 2, intersection_y, intersection_z])
         '''   
        if (15 in detected_ids and 11 in detected_ids): #or (10 in detected_ids and 8 in detected_ids):
            if 15 in detected_ids and 11 in detected_ids:
                marker_1_index = np.where(ids == 15)[0][0]
                marker_2_index = np.where(ids == 11)[0][0]
            '''else:
                marker_1_index = np.where(ids == 10)[0][0]
                marker_2_index = np.where(ids == 8)[0][0]
            marker_1 = tvecs[marker_1_index]
            marker_2 = tvecs[marker_2_index]
            '''
            print("marker1", marker_1,"marker2", marker_2)
            # find the intersection point
            marker_1_y = -marker_1[0][1]
            marker_2_y = -marker_2[0][1]
            intersection_x = ((marker_2_y - marker_1_y) * marker_1[0][0] + (marker_1_y * marker_2[0][0] - marker_2_y * marker_1[0][0])) / (marker_2_y - marker_1_y)
            intersection_z = ((marker_2_y - marker_1_y) * marker_1[0][2] + (marker_1_y * marker_2[0][2] - marker_2_y * marker_1[0][2])) / (marker_2_y - marker_1_y)
            intersection = np.array([intersection_x, (marker_1_y + marker_2_y) / 2, intersection_z])
            
            print("ids: ",ids, end=' ')
            print("intersect: ", intersection)
            image_points = cv2.projectPoints(intersection, rvecs[marker_1_index], tvecs[marker_1_index], cameraMatrix, distCoeffs)[0][0]
            print(image_points)
            
            # Draw a circle at the intersection point (assuming we have an image called 'frame')
            cv2.circle(scaled_frame, (int(np.array(image_points[0][0])), int(np.array(image_points[0][1]))), 5, (255, 0, 255), -1)

        # Draw the detected aruco markers and axes
        ee = cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        for i in range(len(ids)):
            ee = cv2.drawFrameAxes(ee, cameraMatrix, distCoeffs, rvecs[i], tvecs[i], 0.5, thickness=2)


    # Display the resulting frame
    cv2.imshow('frame', ee)


    # Exit loop if 'esc' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break
# Release the video capture object and close all windows
inputVideo.release()
cv2.destroyAllWindows()

True
marker1 [[-0.8807724   0.02964695  4.89559468]] marker2 [[-1.59931106  0.5873147   5.48021093]]
ids:  [[ 3]
 [15]
 [11]] intersect:  [-0.03819923 -0.30848083  0.0310796 ]
[[272.06877595 350.16827103]]
marker1 [[-0.8807724   0.02964695  4.89559468]] marker2 [[-1.59931106  0.5873147   5.48021093]]
ids:  [[ 3]
 [15]
 [11]] intersect:  [-0.03819923 -0.30848083  0.0310796 ]
[[272.06877595 350.16827103]]
marker1 [[-0.8807724   0.02964695  4.89559468]] marker2 [[-1.59931106  0.5873147   5.48021093]]
ids:  [[ 3]
 [15]
 [11]] intersect:  [-0.03819923 -0.30848083  0.0310796 ]
[[272.06877595 350.16827103]]
marker1 [[-0.8807724   0.02964695  4.89559468]] marker2 [[-1.59931106  0.5873147   5.48021093]]
ids:  [[ 3]
 [15]
 [11]] intersect:  [-0.03819923 -0.30848083  0.0310796 ]
[[272.06877595 350.16827103]]
marker1 [[-0.8807724   0.02964695  4.89559468]] marker2 [[-1.59931106  0.5873147   5.48021093]]
ids:  [[ 3]
 [15]
 [11]] intersect:  [-0.03819923 -0.30848083  0.0310796 ]
[[272.06877595 350.1

In [ ]:
import numpy as np

def create_second_point(rvec, tvec, distance):
    rot_mat, _ = cv2.Rodrigues(rvec)
    x_axis = rot_mat[:, 0]
    displacement = x_axis * distance
    second_point = tvec + displacement
    return second_point.flatten()

array([0.1, 0. , 0. , 0.1, 0. , 0. , 0.1, 0. , 0. ])